Quantum Cryptography for Secure IoT
-

In [3]:
import os
import random
import base64
import logging
import numpy as np
from hashlib import sha256
from Crypto.Cipher import AES
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend

# 🔹 Configure Logging
logging.basicConfig(filename="chat_log.txt", level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")

# 🔹 Generate Random Bits (BB84 QKD Simulation)
def generate_random_bits(n): return np.random.randint(2, size=n)
def generate_random_bases(n): return np.random.randint(2, size=n)
def measure_qubits(bits, sender_bases, receiver_bases):
    return np.array([bits[i] if sender_bases[i] == receiver_bases[i] else np.random.randint(2) for i in range(len(bits))])
def reconcile_keys(bits, sender_bases, receiver_bases):
    return bits[np.where(sender_bases == receiver_bases)[0]]

# 🔹 AES-GCM Encryption for Messages
def encrypt_message(message, key):
    cipher = AES.new(key, AES.MODE_GCM)
    nonce = cipher.nonce
    ciphertext, tag = cipher.encrypt_and_digest(message.encode())
    encrypted_data = base64.b64encode(nonce + tag + ciphertext).decode()

    logging.info(f"Message Encrypted: {message} → {encrypted_data}")
    return encrypted_data

# 🔹 AES-GCM Decryption for Messages
def decrypt_message(encrypted_data, key):
    encrypted_data = base64.b64decode(encrypted_data)
    nonce, tag, ciphertext = encrypted_data[:16], encrypted_data[16:32], encrypted_data[32:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    try:
        decrypted_text = cipher.decrypt_and_verify(ciphertext, tag).decode()
        logging.info(f"Message Decrypted: {encrypted_data} → {decrypted_text}")
        return decrypted_text
    except Exception as e:
        logging.error(f"Decryption Failed! Error: {str(e)}")
        return "[ERROR] Decryption Failed"

# Simulate BB84 QKD Key Exchange
n = 64  
sender_bits = generate_random_bits(n)
sender_bases = generate_random_bases(n)
receiver_bases = generate_random_bases(n)

receiver_measured_bits = measure_qubits(sender_bits, sender_bases, receiver_bases)
final_shared_key_sender = reconcile_keys(sender_bits, sender_bases, receiver_bases)
final_shared_key_receiver = reconcile_keys(receiver_measured_bits, sender_bases, receiver_bases)

# Convert final key to hashed format
final_shared_key_str = ''.join(map(str, final_shared_key_sender))
hashed_key = sha256(final_shared_key_str.encode()).digest()

# 🔹 Use HKDF for Key Expansion (AES-128 Key)
hkdf = HKDF(
    algorithm=hashes.SHA256(),
    length=16,
    salt=None,
    info=b'quantum-key',
    backend=default_backend()
)
aes_key = hkdf.derive(hashed_key)

# 🔹 Generate 10 Random Keys (One Correct)
keys = [os.urandom(32).hex() for _ in range(10)]
correct_index = random.randint(0, 9)  # Randomly place the correct key
keys[correct_index] = hashed_key.hex()

# 🔹 AES-GCM Encryption/Decryption for Index
def encrypt_index(index, key):
    cipher = AES.new(key, AES.MODE_GCM)
    nonce = cipher.nonce
    encrypted_index, tag = cipher.encrypt_and_digest(index.to_bytes(1, 'big'))
    return base64.b64encode(nonce + tag + encrypted_index).decode()

def decrypt_index(encrypted_data, key):
    encrypted_data = base64.b64decode(encrypted_data)
    nonce, tag, ciphertext = encrypted_data[:16], encrypted_data[16:32], encrypted_data[32:]
    cipher = AES.new(key, AES.MODE_GCM, nonce=nonce)
    
    try:
        decrypted_index = int.from_bytes(cipher.decrypt_and_verify(ciphertext, tag), 'big')
        logging.info(f"Index Decrypted: {encrypted_data} → {decrypted_index}")
        return decrypted_index
    except Exception as e:
        logging.error(f"Index Decryption Failed! Error: {str(e)}")
        return None

# Encrypt the correct index
encrypted_index = encrypt_index(correct_index, aes_key)

# 🔹 Display Encrypted Index and Key List
print("\n🚀 Secure Encrypted Chat Initialized...\n")
print(f"\n🔒 Encrypted Index (Copy this for decryption): {encrypted_index}")
print("---------------------------------------------------------------------------------------------------------------------")

# 🔹 Shuffle and Display the Keys
random.shuffle(keys)
for i, key in enumerate(keys):
    print(f"{i+1}. {key}")

print("---------------------------------------------------------------------------------------------------------------------")

# 🔹 User Inputs Encrypted Index and Decrypts It
user_encrypted_index = input("\n🔑 Enter the encrypted index: ").strip()
decrypted_index = decrypt_index(user_encrypted_index, aes_key)

if decrypted_index is None:
    print("\n❌ Decryption Failed! Invalid Index.")
    exit()

print(f"\n🔢 Decrypted Index: {decrypted_index + 1} (Pick this key from the list!)")

# 🔹 User Enters Correct Shared Key
user_input_key = input("\n🔑 Enter the correct shared key (copy-paste from key list): ").strip()

if user_input_key == keys[decrypted_index]:
    print("\n✅ Key Verified! Secure chat established. 🔐")
    logging.info("Key Successfully Verified! Secure Chat Started.")
else:
    print("\n❌ Incorrect Key! Access Denied.")
    logging.warning("Incorrect Key Entered! Access Denied.")
    raise SystemExit  # ✅ Ensures program stops correctly

# Initialize chat session
chat_count = 0
max_chats = random.randint(3, 6)  # Key gets destroyed after a few messages
turn = "Sender"  # Sender starts first

while True:
    message = input(f"\n📝 {turn}, enter message (or type 'exit' to quit): ").strip()
    
    if message.lower() == "exit" or chat_count >= max_chats:
        print("\n🔐 Secure Key Destroyed. Chat Ended. 🚀")
        logging.info("Secure Key Destroyed. Chat Session Ended.")
        print("---------------------------------------------------------------------------------------------------------------------")
        
        # 🔎 Debug Information (Displayed at End)
        print(f"\n🔎 DEBUG INFO:")
        print(f"🔑 Correct Hashed Key: {keys[decrypted_index]}")
        print(f"🔢 Correct Key Index: {decrypted_index + 1}")
        print(f"🔑 User Entered Key:   {user_input_key}")
        print("---------------------------------------------------------------------------------------------------------------------")
        
        del keys[decrypted_index]  # Securely delete the key
        break

    # Encrypt and display message
    encrypted_message = encrypt_message(message, aes_key)
    print(f"\n📤 {turn} Encrypted Message: {encrypted_message}")

    # Simulate receiver decrypting the message
    decrypted_message = decrypt_message(encrypted_message, aes_key)
    print(f"📩 {turn} Decrypted Message: {decrypted_message}")

    # Log message exchange
    logging.info(f"{turn} sent encrypted message: {encrypted_message}")
    logging.info(f"{turn} received decrypted message: {decrypted_message}")

    # Alternate turn between Sender and Receiver
    turn = "Receiver" if turn == "Sender" else "Sender"

    chat_count += 1  # Increase chat count
    print("---------------------------------------------------------------------------------------------------------------------")



🚀 Secure Encrypted Chat Initialized...


🔒 Encrypted Index (Copy this for decryption): cWdPbPDvqHrFKRNgMui8WBZODcvBpH2l1J0IJbTUXOH5
---------------------------------------------------------------------------------------------------------------------
1. 1575fc3b206f815ac7dfa864829dce71aca379b22de7c0b7d30fe855dd148716
2. 8c9024b2fde46b316669d276987bc50ea02479e5ad4d2df23a5e3e86fc4f45eb
3. e71f3ff95225f35ce662fae615b56af7c4104800f2ba0ef6cb75367800d9b9c3
4. a2aaeb3d12cac6dd68f02e44c635186b5554581def641ed257280c47514c25d0
5. 50e2cbca94b2f7b214fdc84d30984c0203b9a94a4d53a6eacae4546261c2c4da
6. 36d89a9abb91a5026af06bc471f3a926e163ccdb3eeca10972805a33d7617fbe
7. 0cf4cdd9fa3ced381f5164b89a18b95f8e4922c94f4a7f230dc7b129a08a872f
8. 1fc62466278c62377f1c2fdf0af465e18dafd0c307dd5ec4d4cca8a542050a1f
9. 2d30e3020d6690921da9ab948f3b7a13360f2762bd76d6f4f81b7acb28f86368
10. bb91a5c33f403ebc61a07a45b1d33185bd2b74c523f33221ad9cf4b1f878f365
--------------------------------------------------------------------


🔑 Enter the encrypted index:  cWdPbPDvqHrFKRNgMui8WBZODcvBpH2l1J0IJbTUXOH5



🔢 Decrypted Index: 7 (Pick this key from the list!)



🔑 Enter the correct shared key (copy-paste from key list):  0cf4cdd9fa3ced381f5164b89a18b95f8e4922c94f4a7f230dc7b129a08a872f



✅ Key Verified! Secure chat established. 🔐


...


📤 Receiver Encrypted Message: ulJbUuYDIyQyE2tiGxeIfGcDHGxYR8K/eSpOuB+1WWyVWcsgHg==
📩 Receiver Decrypted Message: Yes ?
---------------------------------------------------------------------------------------------------------------------



📝 Sender, enter message (or type 'exit' to quit):  The otp is 12345, ack the next reply by destroying the key.



📤 Sender Encrypted Message: InySo+Gnk8oel8/rFmCSnV7CTCYU57tudD5+buAIxQrXn24sQRMKIWjyl69RVnk9UieGkSLJUzFw1GXGrRv2c8OBqN3rBIxsYRJdMVd55hWbNaZ/nvOvJpp3Dw==
📩 Sender Decrypted Message: The otp is 12345, ack the next reply by destroying the key.
---------------------------------------------------------------------------------------------------------------------



📝 Receiver, enter message (or type 'exit' to quit):  exit



🔐 Secure Key Destroyed. Chat Ended. 🚀
---------------------------------------------------------------------------------------------------------------------

🔎 DEBUG INFO:
🔑 Correct Hashed Key: 0cf4cdd9fa3ced381f5164b89a18b95f8e4922c94f4a7f230dc7b129a08a872f
🔢 Correct Key Index: 7
🔑 User Entered Key:   0cf4cdd9fa3ced381f5164b89a18b95f8e4922c94f4a7f230dc7b129a08a872f
---------------------------------------------------------------------------------------------------------------------
